In [38]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [39]:
# 获得mnist数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../Resources/mnistdata", one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
print(trainimg.shape, trainlabel.shape)
print(testimg.shape, testlabel.shape)

Extracting ../Resources/mnistdata\train-images-idx3-ubyte.gz
Extracting ../Resources/mnistdata\train-labels-idx1-ubyte.gz
Extracting ../Resources/mnistdata\t10k-images-idx3-ubyte.gz
Extracting ../Resources/mnistdata\t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10)
(10000, 784) (10000, 10)


In [40]:
# params
learning_rate = 0.001
train_epochs = 100
display_step = 10
batch_size = 128

n_features = 784
n_classes = 10
n_hidden1 = 512
n_hidden2 = 512
n_hidden3 = 256

# placeholder
x = tf.placeholder(tf.float32, [None, n_features])
y = tf.placeholder(tf.float32, [None, n_classes])
dropout_keep_prob = tf.placeholder('float')

# variables
stddev = 0.5
weights = {
    'h1' : tf.Variable(tf.random_normal([n_features, n_hidden1], stddev=stddev)),
    'h2' : tf.Variable(tf.random_normal([n_hidden1, n_hidden2], stddev=stddev)),
    'h3' : tf.Variable(tf.random_normal([n_hidden2, n_hidden3], stddev=stddev)),
    'out' : tf.Variable(tf.random_normal([n_hidden3, n_classes], stddev=stddev))
}

biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden1])),
    'b2' : tf.Variable(tf.random_normal([n_hidden2])),
    'b3' : tf.Variable(tf.random_normal([n_hidden3])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}
print("network ready")

network ready


In [41]:
def predit(x, weight, biases, dropout_keep_prob):
    x1 = tf.nn.relu(tf.matmul(x, weights['h1']) + biases['b1'])
    layer1 = x1
    x2 = tf.nn.relu(tf.matmul(layer1, weights['h2']) + biases['b2'])
    layer2 = x2
    x3 = tf.nn.relu(tf.matmul(layer2, weights['h3']) + biases['b3'])
    # use dropout
    layer3 = tf.nn.dropout(x3, dropout_keep_prob)
    return (tf.matmul(layer3, weights['out']) + biases['out'])
    

In [42]:
pred = predit(x, weights, biases, dropout_keep_prob)
# 损失函数 交叉熵
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
# 计算精度
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

init = tf.global_variables_initializer()
print('ready')

ready


In [43]:
# 开始训练
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(train_epochs):
        avg_cost = 0
        n_batch = int(mnist.train.num_examples / batch_size)
        for i in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feeds = {x: batch_xs, y: batch_ys, dropout_keep_prob: 0.6}
            sess.run(optimizer, feed_dict=feeds)
            feeds = {x: batch_xs, y: batch_ys, dropout_keep_prob: 1.0}
            avg_cost += sess.run(cost, feed_dict=feeds)
        avg_cost /= n_batch
        
        # 显示
        if epoch % display_step == 0:
            print ("Epoch: %03d/%03d cost: %.9f" % (epoch, train_epochs, avg_cost))
            feeds = {x: batch_xs, y: batch_ys, dropout_keep_prob: 1.0}
            train_acc = sess.run(accr, feed_dict=feeds)
            print ("TRAIN ACCURACY: %.3f" % (train_acc))
            feeds = {x: testimg, y: testlabel, dropout_keep_prob: 1.0}
            test_acc = sess.run(accr, feed_dict=feeds)
            print ("TEST ACCURACY: %.3f" % (test_acc))
            
        

Epoch: 000/100 cost: 89.761417199
TRAIN ACCURACY: 0.844
TEST ACCURACY: 0.818
Epoch: 010/100 cost: 0.956600005
TRAIN ACCURACY: 0.656
TEST ACCURACY: 0.660
Epoch: 020/100 cost: 0.575819413
TRAIN ACCURACY: 0.789
TEST ACCURACY: 0.815
Epoch: 030/100 cost: 0.363385069
TRAIN ACCURACY: 0.898
TEST ACCURACY: 0.906
Epoch: 040/100 cost: 0.237642780
TRAIN ACCURACY: 0.930
TEST ACCURACY: 0.926
Epoch: 050/100 cost: 0.159606950
TRAIN ACCURACY: 0.969
TEST ACCURACY: 0.941
Epoch: 060/100 cost: 0.114717439
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.957
Epoch: 070/100 cost: 0.098926095
TRAIN ACCURACY: 0.953
TEST ACCURACY: 0.956
Epoch: 080/100 cost: 0.073473029
TRAIN ACCURACY: 0.961
TEST ACCURACY: 0.963
Epoch: 090/100 cost: 0.052092432
TRAIN ACCURACY: 0.992
TEST ACCURACY: 0.967
